In [1]:
import py_vncorenlp
py_vncorenlp.download_model(save_dir='/workspace/nlplab/kienvt/COQE/pretrain_model/vncorenlp')

VnCoreNLP model folder /workspace/nlplab/kienvt/COQE/pretrain_model/vncorenlp already exists! Please load VnCoreNLP from this folder!


In [2]:
model = py_vncorenlp.VnCoreNLP(save_dir='/workspace/nlplab/kienvt/COQE/pretrain_model/vncorenlp')

In [3]:
def get_segment_idx_map(src_tokens_col, trg_tokens_col):
    final_result = []
    assert len(src_tokens_col) == len(trg_tokens_col)
    for index in range(len(src_tokens_col)):
        rawtoken2id = src_tokens_col[index]
        segmenttoken2id = trg_tokens_col[index]
        result = []
        id_raw = 1
        id_segment = 1
        src_cache = ''
        for token, id in segmenttoken2id:
            tokens = token.split("_")
            segment_tokens_set = []
            # print(tokens, id_segment, id_raw)
            for tok in tokens:
                src_cache += tok.strip()
                # print(src_cache.strip(), rawtoken2id[id_raw-1][0])
                if src_cache.strip() == rawtoken2id[id_raw-1][0] \
                        or len(src_cache.strip()) == len(rawtoken2id[id_raw-1][0]):
                    segment_tokens_set.append(id_raw)
                    id_raw += 1
                    src_cache = ''
            if segment_tokens_set != []:
                result.append((id_segment, segment_tokens_set))
                id_segment += 1
        final_result.append(result)

    return final_result

---

In [4]:
%cd /workspace/nlplab/kienvt/COQE/multi_stage_approach

/workspace/nlplab/kienvt/COQE/multi_stage_approach


In [5]:
from data_utils import current_program_code as cpc
from data_utils import shared_utils, label_parse

/workspace/nlplab/kienvt/COQE/cenv/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [56]:
data_type = 'dev'

In [57]:
sent_col, sent_label_col, final_label_col = cpc.read_standard_file(f"/workspace/nlplab/kienvt/COQE/data/VLSP23-Addons/{data_type}.txt")

In [58]:
sent_col

['title : So sánh Samsung Galaxy S22 vs Galaxy S21 : đâu là những thay đổi đáng mong đợi ? ',
 'Samsung Galaxy S22 sẽ không có quá nhiều sự khác biệt với phiên bản tiền nhiệm ',
 'Khi bạn đọc bài viết này , thời điểm Samsung Galaxy S22 chính thức ra mắt chỉ còn khoảng vài tiếng đồng hồ nữa . ',
 'Và chúng ta hãy cùng nhau điểm lại những thay đổi đáng chú ý nhất về bản tiêu chuẩn của dòng điện thoại này so với người tiền nhiệm Galaxy S21 . ',
 'alt : Samsung Galaxy S22 có rất nhiều điều khác biệt so với thế hệ S21 như màn hình ',
 'des : ',
 'Kích thước nhỏ gọn và thuận tiện hơn ',
 'Theo tiết lộ từ nhiều trang báo , Samsung Galaxy S22 sẽ không có sự thay đổi về mặt thiết kế so với phiên bản 2021 , thay vào đó Samsung muốn tập trung nhiều hơn về kích thước để đem đến một trải nghiệm tối ưu dành cho người dùng . ',
 'Theo đó kích thước của mẫu điện thoại này sẽ là 146 x 70.6 x 7.6 mm , nếu so với thông số của S21 ( 151.7 x 71.2 x 7.9 mm ) thì có thể thấy ngay rằng phiên bản 2022 đã được 

In [59]:
final_label_col

[['[[];[];[];[];[]]'],
 ['[[1&&Samsung 2&&Galaxy 3&&S22];[13&&phiên 14&&bản 15&&tiền 16&&nhiệm];[];[5&&không 6&&có 7&&quá 8&&nhiều 9&&sự 10&&khác 11&&biệt];[EQL]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[1&&Kích 2&&thước];[3&&nhỏ 4&&gọn 5&&và 6&&thuận 7&&tiện 8&&hơn];[COM+]]'],
 ['[[9&&Samsung 10&&Galaxy 11&&S22];[24&&phiên 25&&bản 26&&2021];[20&&thiết 21&&kế];[13&&không 14&&có 15&&sự 16&&thay 17&&đổi];[EQL]]'],
 ['[[40&&phiên 41&&bản 42&&2022];[25&&S21];[];[49&&ngắn 50&&và 51&&mỏng 52&&hơn 53&&đáng 54&&kể];[COM]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[6&&cảm 7&&giác 8&&cầm 9&&nắm];[21&&thuận 22&&tiện 23&&hơn 24&&rất 25&&nhiều];[COM+]]',
  '[[];[];[11&&sử 12&&dụng 13&&một 14&&tay];[21&&thuận 22&&tiện 23&&hơn 24&&rất 25&&nhiều];[COM+]]'],
 ['[[13&&S22];[19&&bản 20&&tiền 21&&nhiệm];[10&&màn 11&&hình];[15&&nhỏ 16&&hơn];[COM-]]'],
 ['[[];[];[];[];[]]'],
 ['[[3&&Galaxy 4&&S22];[];[1&&Camera];[13&&thay 14&&

Train: 
+ câu "title" mang nhãn so sánh (0 / 40 = 0%) 
+ câu "alt" / "des" mang nhãn so sánh (20 / 672 = 3%)
+ câu len < 20 mang nhãn (0 / 402 = 0%), < 30 (3 / 513 = 0,6%), < 50 (27 / 841 = 3%)

Dev: 
+ câu "title" mang nhãn so sánh (0 / 12 = 0%) 
+ câu "alt" / "des" mang nhãn so sánh (3 / 244 = 1%)
+ câu len < 20 mang nhãn (0 / 126 = 0%), < 30 (0 / 167 = 0%), < 50 (12 / 316 = 3,7%)

Test: 
+ câu "title" mang nhãn so sánh (1 / 5 = 17%) 
+ câu "alt" / "des" mang nhãn so sánh (2 / 82 = 2,4%)
+ câu len < 20 mang nhãn (0 / 51 = 0%), < 30 (0 / 62 = 0%), < 50 (1 / 96 = 1%)


In [60]:
result_label_unique = ['[DIF]', '[EQL]', '[COM+]', '[COM-]', '[COM]']
label_col_1hot = []

for item in final_label_col:
    new_item = []
    for subitem in item:
        tmp = subitem
        for idx, label in enumerate(result_label_unique):
            tmp = tmp.replace(label, f'[{idx}]')
        new_item.append(tmp)
    label_col_1hot.append(new_item)

label_col_1hot

[['[[];[];[];[];[]]'],
 ['[[1&&Samsung 2&&Galaxy 3&&S22];[13&&phiên 14&&bản 15&&tiền 16&&nhiệm];[];[5&&không 6&&có 7&&quá 8&&nhiều 9&&sự 10&&khác 11&&biệt];[1]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[1&&Kích 2&&thước];[3&&nhỏ 4&&gọn 5&&và 6&&thuận 7&&tiện 8&&hơn];[2]]'],
 ['[[9&&Samsung 10&&Galaxy 11&&S22];[24&&phiên 25&&bản 26&&2021];[20&&thiết 21&&kế];[13&&không 14&&có 15&&sự 16&&thay 17&&đổi];[1]]'],
 ['[[40&&phiên 41&&bản 42&&2022];[25&&S21];[];[49&&ngắn 50&&và 51&&mỏng 52&&hơn 53&&đáng 54&&kể];[4]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[];[];[]]'],
 ['[[];[];[6&&cảm 7&&giác 8&&cầm 9&&nắm];[21&&thuận 22&&tiện 23&&hơn 24&&rất 25&&nhiều];[2]]',
  '[[];[];[11&&sử 12&&dụng 13&&một 14&&tay];[21&&thuận 22&&tiện 23&&hơn 24&&rất 25&&nhiều];[2]]'],
 ['[[13&&S22];[19&&bản 20&&tiền 21&&nhiệm];[10&&màn 11&&hình];[15&&nhỏ 16&&hơn];[3]]'],
 ['[[];[];[];[];[]]'],
 ['[[3&&Galaxy 4&&S22];[];[1&&Camera];[13&&thay 14&&đổi 15&&lớn];[0]]'

In [61]:
len(final_label_col) == len(label_col_1hot)

True

In [62]:
len([sl for sl in sent_label_col if sl == 1]) / len(sent_label_col)

0.25193370165745854

Số câu mang nhãn so sánh: 20%

In [63]:
LP = label_parse.LabelParser(label_col_1hot, ["entity_1", "entity_2", "aspect", "result"])

In [64]:
label_col, tuple_pair_col = LP.parse_sequence_label("&&", sent_col, file_type="eng")

In [65]:
label_col

[{'entity_1': set(), 'entity_2': set(), 'aspect': set(), 'result': set()},
 {'entity_1': {(0, 3)},
  'entity_2': {(12, 16)},
  'aspect': set(),
  'result': {(4, 11, 1)}},
 {'entity_1': set(), 'entity_2': set(), 'aspect': set(), 'result': set()},
 {'entity_1': set(), 'entity_2': set(), 'aspect': set(), 'result': set()},
 {'entity_1': set(), 'entity_2': set(), 'aspect': set(), 'result': set()},
 {'entity_1': set(), 'entity_2': set(), 'aspect': set(), 'result': set()},
 {'entity_1': set(),
  'entity_2': set(),
  'aspect': {(0, 2)},
  'result': {(2, 8, 2)}},
 {'entity_1': {(8, 11)},
  'entity_2': {(23, 26)},
  'aspect': {(19, 21)},
  'result': {(12, 17, 1)}},
 {'entity_1': {(39, 42)},
  'entity_2': {(24, 25)},
  'aspect': set(),
  'result': {(48, 54, 4)}},
 {'entity_1': set(), 'entity_2': set(), 'aspect': set(), 'result': set()},
 {'entity_1': set(), 'entity_2': set(), 'aspect': set(), 'result': set()},
 {'entity_1': set(),
  'entity_2': set(),
  'aspect': {(5, 9), (10, 14)},
  'result': {

In [66]:
tuple_pair_col

[[[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(0, 3), (12, 16), (-1, -1), (4, 11), (1, 1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (0, 2), (2, 8), (2, 2)]],
 [[(8, 11), (23, 26), (19, 21), (12, 17), (1, 1)]],
 [[(39, 42), (24, 25), (-1, -1), (48, 54), (4, 4)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (5, 9), (20, 25), (2, 2)],
  [(-1, -1), (-1, -1), (10, 14), (20, 25), (2, 2)]],
 [[(12, 13), (18, 21), (9, 11), (14, 16), (3, 3)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(2, 4), (-1, -1), (0, 1), (12, 15), (0, 0)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1),

---

In [67]:
segment_sent_col = [" ".join(model.word_segment(s)) for s in sent_col]
segment_sent_col

['title : So_sánh Samsung Galaxy S22 vs Galaxy S21 : đâu là những thay_đổi đáng mong_đợi ?',
 'Samsung Galaxy S22 sẽ không có quá nhiều sự khác_biệt với phiên_bản tiền_nhiệm',
 'Khi bạn_đọc bài viết này , thời_điểm Samsung Galaxy S22 chính_thức ra_mắt chỉ còn khoảng vài tiếng đồng_hồ nữa .',
 'Và chúng_ta hãy cùng nhau điểm lại những thay_đổi đáng chú_ý nhất về bản tiêu_chuẩn của dòng điện_thoại này so với người tiền_nhiệm Galaxy S21 .',
 'alt : Samsung Galaxy S22 có rất nhiều điều khác_biệt so với thế_hệ S21 như màn_hình',
 'des :',
 'Kích_thước nhỏ gọn và thuận_tiện hơn',
 'Theo tiết_lộ từ nhiều trang báo , Samsung Galaxy S22 sẽ không có sự thay_đổi về mặt thiết_kế so với phiên_bản 2021 , thay vào đó Samsung muốn tập_trung nhiều hơn về kích_thước để đem đến một trải_nghiệm tối_ưu dành cho người dùng .',
 'Theo đó kích_thước của mẫu điện_thoại này sẽ là 146 x 70.6 x 7.6 mm , nếu so với thông_số của S21 ( 151.7 x 71.2 x 7.9 mm ) thì có_thể thấy ngay rằng phiên_bản 2022 đã được thu nhỏ 

In [68]:
raw_token2id_col = [[(token, i+1) for i, token in zip(range(len(s.split(" "))), s.split(" ")) if token != ''] \
                                                     for s in sent_col]
raw_token2id_col

[[('title', 1),
  (':', 2),
  ('So', 3),
  ('sánh', 4),
  ('Samsung', 5),
  ('Galaxy', 6),
  ('S22', 7),
  ('vs', 8),
  ('Galaxy', 9),
  ('S21', 10),
  (':', 11),
  ('đâu', 12),
  ('là', 13),
  ('những', 14),
  ('thay', 15),
  ('đổi', 16),
  ('đáng', 17),
  ('mong', 18),
  ('đợi', 19),
  ('?', 20)],
 [('Samsung', 1),
  ('Galaxy', 2),
  ('S22', 3),
  ('sẽ', 4),
  ('không', 5),
  ('có', 6),
  ('quá', 7),
  ('nhiều', 8),
  ('sự', 9),
  ('khác', 10),
  ('biệt', 11),
  ('với', 12),
  ('phiên', 13),
  ('bản', 14),
  ('tiền', 15),
  ('nhiệm', 16)],
 [('Khi', 1),
  ('bạn', 2),
  ('đọc', 3),
  ('bài', 4),
  ('viết', 5),
  ('này', 6),
  (',', 7),
  ('thời', 8),
  ('điểm', 9),
  ('Samsung', 10),
  ('Galaxy', 11),
  ('S22', 12),
  ('chính', 13),
  ('thức', 14),
  ('ra', 15),
  ('mắt', 16),
  ('chỉ', 17),
  ('còn', 18),
  ('khoảng', 19),
  ('vài', 20),
  ('tiếng', 21),
  ('đồng', 22),
  ('hồ', 23),
  ('nữa', 24),
  ('.', 25)],
 [('Và', 1),
  ('chúng', 2),
  ('ta', 3),
  ('hãy', 4),
  ('cùng', 5),
 

In [69]:
segment_token2id_col = [[(token, i+1) for i, token in zip(range(len(s.split(" "))), s.split(" ")) if token != ''] \
                                                                            for s in segment_sent_col]
segment_token2id_col

[[('title', 1),
  (':', 2),
  ('So_sánh', 3),
  ('Samsung', 4),
  ('Galaxy', 5),
  ('S22', 6),
  ('vs', 7),
  ('Galaxy', 8),
  ('S21', 9),
  (':', 10),
  ('đâu', 11),
  ('là', 12),
  ('những', 13),
  ('thay_đổi', 14),
  ('đáng', 15),
  ('mong_đợi', 16),
  ('?', 17)],
 [('Samsung', 1),
  ('Galaxy', 2),
  ('S22', 3),
  ('sẽ', 4),
  ('không', 5),
  ('có', 6),
  ('quá', 7),
  ('nhiều', 8),
  ('sự', 9),
  ('khác_biệt', 10),
  ('với', 11),
  ('phiên_bản', 12),
  ('tiền_nhiệm', 13)],
 [('Khi', 1),
  ('bạn_đọc', 2),
  ('bài', 3),
  ('viết', 4),
  ('này', 5),
  (',', 6),
  ('thời_điểm', 7),
  ('Samsung', 8),
  ('Galaxy', 9),
  ('S22', 10),
  ('chính_thức', 11),
  ('ra_mắt', 12),
  ('chỉ', 13),
  ('còn', 14),
  ('khoảng', 15),
  ('vài', 16),
  ('tiếng', 17),
  ('đồng_hồ', 18),
  ('nữa', 19),
  ('.', 20)],
 [('Và', 1),
  ('chúng_ta', 2),
  ('hãy', 3),
  ('cùng', 4),
  ('nhau', 5),
  ('điểm', 6),
  ('lại', 7),
  ('những', 8),
  ('thay_đổi', 9),
  ('đáng', 10),
  ('chú_ý', 11),
  ('nhất', 12),
  ('

In [70]:
segment_idx_map = get_segment_idx_map(raw_token2id_col, segment_token2id_col)

In [71]:
segment_idx_map

[[(1, [1]),
  (2, [2]),
  (3, [3, 4]),
  (4, [5]),
  (5, [6]),
  (6, [7]),
  (7, [8]),
  (8, [9]),
  (9, [10]),
  (10, [11]),
  (11, [12]),
  (12, [13]),
  (13, [14]),
  (14, [15, 16]),
  (15, [17]),
  (16, [18, 19]),
  (17, [20])],
 [(1, [1]),
  (2, [2]),
  (3, [3]),
  (4, [4]),
  (5, [5]),
  (6, [6]),
  (7, [7]),
  (8, [8]),
  (9, [9]),
  (10, [10, 11]),
  (11, [12]),
  (12, [13, 14]),
  (13, [15, 16])],
 [(1, [1]),
  (2, [2, 3]),
  (3, [4]),
  (4, [5]),
  (5, [6]),
  (6, [7]),
  (7, [8, 9]),
  (8, [10]),
  (9, [11]),
  (10, [12]),
  (11, [13, 14]),
  (12, [15, 16]),
  (13, [17]),
  (14, [18]),
  (15, [19]),
  (16, [20]),
  (17, [21]),
  (18, [22, 23]),
  (19, [24]),
  (20, [25])],
 [(1, [1]),
  (2, [2, 3]),
  (3, [4]),
  (4, [5]),
  (5, [6]),
  (6, [7]),
  (7, [8]),
  (8, [9]),
  (9, [10, 11]),
  (10, [12]),
  (11, [13, 14]),
  (12, [15]),
  (13, [16]),
  (14, [17]),
  (15, [18, 19]),
  (16, [20]),
  (17, [21]),
  (18, [22, 23]),
  (19, [24]),
  (20, [25]),
  (21, [26]),
  (22, [27]

In [72]:
len(segment_idx_map)

905

---

In [73]:
segment_tuple_pair_col = []
for i in range(len(tuple_pair_col)):
    segmentid_2_rawids = segment_idx_map[i]
    segment_label_set = []
    for label_set in tuple_pair_col[i]:
        segment_elem_label_set = []
        for elem_pair in label_set[:-1]:
            if elem_pair != (-1, -1):
                bid = elem_pair[0] + 1
                eid = elem_pair[1]
                new_bid = bid
                new_eid = eid
                exist_new_bid = 0
                exist_new_eid = 0
                for segment_id, raw_ids in segmentid_2_rawids:
                    if bid in raw_ids:
                        new_bid = segment_id
                        exist_new_bid += 1
                    if eid in raw_ids:
                        new_eid = segment_id
                        exist_new_eid += 1
                if exist_new_bid == 0 or exist_new_eid == 0:
                    print(i, (bid, eid))
                    print(segmentid_2_rawids)
                    print(label_set)
                segment_elem_label_set.append((new_bid, new_eid))
                # print((bid, eid))
                # print(segmentid_2_rawids)
                # print((new_bid, new_eid))
                # print('=========')
            else:
                segment_elem_label_set.append((-1, -1))
        segment_elem_label_set.append(label_set[-1])        
        segment_label_set.append(segment_elem_label_set)
    segment_tuple_pair_col.append(segment_label_set)

In [74]:
segment_tuple_pair_col

[[[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(1, 3), (12, 13), (-1, -1), (5, 10), (1, 1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (1, 1), (2, 6), (2, 2)]],
 [[(8, 10), (21, 22), (18, 18), (12, 15), (1, 1)]],
 [[(36, 37), (22, 22), (-1, -1), (44, 48), (4, 4)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (6, 8), (18, 21), (2, 2)],
  [(-1, -1), (-1, -1), (10, 12), (18, 21), (2, 2)]],
 [[(10, 10), (16, 17), (8, 8), (12, 13), (3, 3)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(3, 4), (-1, -1), (1, 1), (12, 13), (0, 0)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), (-1, -1)]],
 [[(-1, -1), (-1, -1), (-1, -1), (-1, -1), 

In [75]:
len(segment_tuple_pair_col)

905

In [76]:
len(segment_token2id_col)

905

---

In [77]:
segment_data = []
error_ids = []
for index in range(len(segment_tuple_pair_col)):
    segmenttoken2id = segment_token2id_col[index]
    id2segmenttoken = {id: tok for tok, id in segmenttoken2id}
    # print(id2segmenttoken)
    label_set = segment_tuple_pair_col[index]
    segment_data.append(" ".join([tok for tok, _ in segmenttoken2id]))
    is_sent = True
    for label_elem in label_set:
        if label_elem[-1] == (-1, -1):
            segment_data[-1] += ' \t0'
            segment_data.append('[[];[];[];[];[]]')
            break
        else:
            if is_sent:
                segment_data[-1] += ' \t1'
                is_sent = False
            new_label_elem = []
            for elem in label_elem[:-1]:
                if elem == (-1, -1):
                    new_label_elem.append('[]')
                else:
                    elem_parse = []
                    for i in range(elem[0], elem[-1]+1):
                        try:
                            # print(elem, i, len(id2segmenttoken))
                            elem_parse.append(f'{i}&&{id2segmenttoken[i]}')
                        except Exception:
                            error_ids.append(index)
                            elem_parse.append('ERROR')
                    new_label_elem.append("[" + " ".join(elem_parse)  + "]")
                    # print(new_label_elem)
            segment_data.append("[" + ";".join(new_label_elem) + \
                                          f';{result_label_unique[label_elem[-1][0]]}' + "]")

In [78]:
print("\n".join(segment_data))

title : So_sánh Samsung Galaxy S22 vs Galaxy S21 : đâu là những thay_đổi đáng mong_đợi ? 	0
[[];[];[];[];[]]
Samsung Galaxy S22 sẽ không có quá nhiều sự khác_biệt với phiên_bản tiền_nhiệm 	1
[[1&&Samsung 2&&Galaxy 3&&S22];[12&&phiên_bản 13&&tiền_nhiệm];[];[5&&không 6&&có 7&&quá 8&&nhiều 9&&sự 10&&khác_biệt];[EQL]]
Khi bạn_đọc bài viết này , thời_điểm Samsung Galaxy S22 chính_thức ra_mắt chỉ còn khoảng vài tiếng đồng_hồ nữa . 	0
[[];[];[];[];[]]
Và chúng_ta hãy cùng nhau điểm lại những thay_đổi đáng chú_ý nhất về bản tiêu_chuẩn của dòng điện_thoại này so với người tiền_nhiệm Galaxy S21 . 	0
[[];[];[];[];[]]
alt : Samsung Galaxy S22 có rất nhiều điều khác_biệt so với thế_hệ S21 như màn_hình 	0
[[];[];[];[];[]]
des : 	0
[[];[];[];[];[]]
Kích_thước nhỏ gọn và thuận_tiện hơn 	1
[[];[];[1&&Kích_thước];[2&&nhỏ 3&&gọn 4&&và 5&&thuận_tiện 6&&hơn];[COM+]]
Theo tiết_lộ từ nhiều trang báo , Samsung Galaxy S22 sẽ không có sự thay_đổi về mặt thiết_kế so với phiên_bản 2021 , thay vào đó Samsung muốn 

In [79]:
import os

file_name = f'/workspace/nlplab/kienvt/COQE/data/VLSP23-Addons-Segmented/{data_type}.txt'

if os.path.exists(file_name):
    os.remove(file_name)

with open(file_name, "a") as file:
    file.write("\n".join(segment_data))
    file.close()

In [80]:
error_ids

[]